In [1]:
from __future__ import division
import os
import re
import random
import time
import binascii
import pprint
import nltk
from bisect import bisect_right
from heapq import heappop, heappush
from nltk.util import ngrams

In [2]:
numHashes = 10
numDocs = 100
dataFile = "./data/articles_" + str(numDocs) + ".train"
truthFile = "./data/articles_" + str(numDocs) + ".truth"

In [3]:
plagiaries = {}

# Open the truth file.
f = open(truthFile, "rU")

# For each line of the files...
for line in f:
  # Strip the newline character, if present.
  if line[-1] == '\n':
      line = line[0:-1]
      
  docs = line.split(" ")

  # Map the two documents to each other.
  plagiaries[docs[0]] = docs[1]
  plagiaries[docs[1]] = docs[0]

# pprint.pprint(plagiaries)

In [23]:
f = open(dataFile, "rU")

for i in range(0, 1):
    # each line is an article, words[0] is article id
    line = f.readline()
    id_article = line.split(' ', 1)
    
    singles = set()
    
#     # Words tokenize
#     tokenize = nltk.word_tokenize(id_article[1])
#     ngram_gen = ngrams(tokenize,5)
#     for grams in ngram_gen:
#         single = ' '.join(grams)
#         singles.add(single)
#         # print single
    
    # Characters tokenize
    tokenize = list(id_article[1])
    ngram_gen = ngrams(tokenize,26)
    for grams in ngram_gen:
        single = ''.join(grams)
        singles.add(single)
        # print single

pprint.pprint(singles)
    

set([' "growth" and the other, m',
     ' "value". I don\'t know if ',
     " -- I haven't seen it. But",
     ' 2, 2002 are moving today ',
     ' 2002 are moving today to ',
     ' Alberto Gonzales would ha',
     ' April 2, 2002 are moving ',
     " Australia's farmers shoul",
     ' Bruni-Sarkozy. These colu',
     ' Bush expressed confidence',
     " But I'm pretty certain th",
     ' Carla Bruni-Sarkozy. Thes',
     ' China later this month, j',
     ' Deputy Prime Minister and',
     ' French President Nicolas ',
     ' George W. Bush expressed ',
     ' Gonzales would have no be',
     " I don't know if ``Harry P",
     " I haven't seen it. But I'",
     " I'm pretty certain that i",
     ' Inc. said Wednesday it ma',
     ' Israeli helicopter in a p',
     ' It is a visual representa',
     ' John Anderson said Tuesda',
     ' Leader John Anderson said',
     ' Media and entertainment g',
     ' Minister and National Par',
     ' Monday about passing an i',
     ' Monday in th

In [ ]:
print "Shingling articles..."

# The current shingle ID value to assign to the next new shingle we 
# encounter. When a shingle gets added to the dictionary, we'll increment this
# value.
curShingleID = 0

# Create a dictionary of the articles, mapping the article identifier (e.g., 
# "t8470") to the list of shingle IDs that appear in the document.
docsAsShingleSets = {};
  
# Open the data file.
f = open(dataFile, "rU")

docNames = []

t0 = time.time()

totalShingles = 0

for i in range(0, numDocs):
  
  # Read all of the words (they are all on one line) and split them by white
  # space.
  words = f.readline().split(" ") 
  
  # Retrieve the article ID, which is the first word on the line.  
  docID = words[0]
  
  # Maintain a list of all document IDs.  
  docNames.append(docID)
    
  del words[0]  
  
  # 'shinglesInDoc' will hold all of the unique shingle IDs present in the 
  # current document. If a shingle ID occurs multiple times in the document,
  # it will only appear once in the set (this is a property of Python sets).
  shinglesInDoc = set()
  
  # For each word in the document...
  for index in range(0, len(words) - 2):
    # Construct the shingle text by combining three words together.
    shingle = words[index] + " " + words[index + 1] + " " + words[index + 2]

    # Hash the shingle to a 32-bit integer.
    crc = binascii.crc32(shingle) & 0xffffffff
    
    # Add the hash value to the list of shingles for the current document. 
    # Note that set objects will only add the value to the set if the set 
    # doesn't already contain it. 
    shinglesInDoc.add(crc)
  
  # Store the completed list of shingles for this document in the dictionary.
  docsAsShingleSets[docID] = shinglesInDoc
  
  # Count the number of shingles across all documents.
  totalShingles = totalShingles + (len(words) - 2)

# Close the data file.  
f.close()  

# Report how long shingling took.
print '\nShingling ' + str(numDocs) + ' docs took %.2f sec.' % (time.time() - t0)
 
print '\nAverage shingles per doc: %.2f' % (totalShingles / numDocs)